In [2]:
import h2o
h2o.init()            #uses all cores by default
h2o.remove_all()      #clean slate, in case cluster was already running

H2O cluster uptime:,8 days 12 hours 53 minutes 56 seconds 387 milliseconds
H2O cluster version:,3.8.2.3
H2O cluster name:,H2O_started_from_python_joe_igy224
H2O cluster total nodes:,1
H2O cluster total free memory:,634.65 MB
H2O cluster total cores:,1
H2O cluster allowed cores:,1
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


In [3]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
import numpy as np

df = h2o.import_file('small_car.csv')
#help(h2o.import_file)
train, test = df.split_frame([0.8], seed=63)
print len(train), len(test)


Parse Progress: [##################################################] 100%
81 19


In [4]:
target_simple_Y = 'Horsepower'
exp_simple_X = 'Displacement'

target_multi_Y = 'Horsepower'
exp_multi_X = ['Cylinders', 'Displacement', 'Manufacturer', 'Model_Year', 'Origin', 'Weight']

In [5]:
glm_simple_cars = H2OGeneralizedLinearEstimator(
                    model_id='glm_simple_cars',  #will appear in Flow by this name
                    family='gaussian', 
                    solver='IRLSM' #'L_BFGS' is the other GLM solveraka
                    ,lambda_search = True
                )  

glm_multi_cars = H2OGeneralizedLinearEstimator(
                    model_id='glm_multi_cars', 
                    family='gaussian', 
                    solver='IRLSM' 
                    ,lambda_search = True
                )  

In [6]:
glm_simple_cars.train(exp_simple_X, target_simple_Y, training_frame=train) #, validation_frame=valid)
glm_multi_cars.train(exp_multi_X, target_multi_Y, training_frame=train) #, validation_frame=valid)


glm Model Build Progress: [##################################################] 100%

glm Model Build Progress: [##################################################] 100%


In [7]:
glm_simple_cars

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  glm_simple_cars

GLM Model: summary



,family,link,regularization,lambda_search,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
,gaussian,identity,"Elastic Net (alpha = 0.5, lambda = 0.008369 )","nlambda = 100, lambda_max = 0.0, best_lambda = 0.008369",2,1,0,py_2




ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 304.011240948
R^2: 0.853623161308
Mean Residual Deviance: 304.011240948
Null degrees of freedom: 80
Residual degrees of freedom: 79
Null deviance: 168229.555556
Residual deviance: 24624.9105168
AIC: 698.950281259

Scoring History: 


,timestamp,duration,iteration,lambda,predictors,explained_deviance_train,explained_deviance_test
,2016-05-10 07:31:15,0.000 sec,0,.84E2,1,0.0000000,nan
,2016-05-10 07:31:15,0.000 sec,0,.76E2,2,0.0038224,nan
,2016-05-10 07:31:15,0.001 sec,0,.69E2,2,0.0079919,nan
,2016-05-10 07:31:15,0.001 sec,0,.63E2,2,0.0125327,nan
,2016-05-10 07:31:15,0.001 sec,0,.58E2,2,0.0174729,nan
---,---,---,---,---,---,---,---
,2016-05-10 07:31:15,0.092 sec,0,.12E-1,2,0.8536057,nan
,2016-05-10 07:31:15,0.094 sec,0,.11E-1,2,0.8536113,nan
,2016-05-10 07:31:15,0.095 sec,0,.1E-1,2,0.8536160,nan
,2016-05-10 07:31:15,0.095 sec,0,.92E-2,2,0.8536199,nan



See the whole table with table.as_data_frame()


In [8]:
#SIMPLE: target = horsepower, explanatory = displacement

#the test set used for lecture 11, a hack since some of these are likely in the training set here
test_simple = h2o.import_file('lec11_test.csv')

print '#my homework had MSE = 602.31 -> RMSE = 24.54'
print 'RMSE: {0}'.format(np.sqrt(glm_simple_cars.mse())) 
glm_simple_pred = glm_simple_cars.predict(test_simple)
#print 'pred object dimensions: {0}'.format(glm_simple_pred.dim)

print("GLM predictions: ")
df_simple = test_simple[['Displacement','Horsepower']].cbind(glm_simple_pred)
print df_simple.head()
py_df_simple = df_simple.as_data_frame(use_pandas=False)
print py_df_simple[:10]
h2o.export_file(frame=df_simple, path='car_SIMPLE.csv', force=True)


Parse Progress: [##################################################] 100%
#my homework had MSE = 602.31 -> RMSE = 24.54
RMSE: 17.4359181275

glm prediction Progress: [##################################################] 100%
GLM predictions: 
  Displacement    Horsepower    predict
--------------  ------------  ---------
           455           225   202.916
           200            85   109.17
           318           210   152.551
           250           105   127.552
            91            53    69.0986
            97            75    71.3043
           112            88    76.8188
           151            90    91.1564
            91            67    69.0986
           232           112   120.935

[10 rows x 3 columns]
[['Displacement', 'Horsepower', 'predict'], ['455', '225', '202.9161870315443'], ['200', '85', '109.17031757223305'], ['318', '210', '152.55075912595356'], ['250', '105', '127.55186060347054'], ['91', '53', '69.09855376413529'], ['97', '75', '71.3043389278838'

In [ ]:
glm_multi_cars

In [9]:
#MULTI: target = hp, explanatory = #cylinders, displacement, manufacturer, model_year, origin and weight 
#Lec11 spark MLLIB: 38.65 RMSE for mutliple -> predict hp 
#SIMPLE above gave an RMSE = 17.4
print 'RMSE: {0}'.format(np.sqrt(glm_multi_cars.mse())) 
glm_multi_pred = glm_multi_cars.predict(test)
print("GLM predictions: ")

df_multi = test[['Displacement','Horsepower']].cbind(glm_multi_pred)
print df_multi.head().apply(fun=lambda row : row) #playing around with
h2o.export_file(frame=df_multi, path='car_MULTI.csv', force=True)

RMSE: 17.8203350844

glm prediction Progress: [##################################################] 100%
GLM predictions: 
  Displacement    Horsepower    predict
--------------  ------------  ---------
           350           165   159.519
           318           150   150.992
           133           115   105.469
           350           165   164.734
            97            88    87.7069
            97            46    83.6093
           199            90   115.448
           318           210   164.132
           200            81   113.163
            90            70    75.7201

[10 rows x 3 columns]

Export File Progress: [##################################################] 100%
